<a href="https://colab.research.google.com/github/manjuiitm/AgenticAI/blob/main/HuggingFace%2CLlama_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!wget https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v0.3-GGUF/resolve/main/tinyllama-1.1b-chat-v0.3.Q4_K_M.gguf -O /content/tinyllama.gguf


--2025-08-24 09:16:06--  https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v0.3-GGUF/resolve/main/tinyllama-1.1b-chat-v0.3.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 18.239.50.16, 18.239.50.80, 18.239.50.103, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.16|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/65/bb/65bb2b45ed3e7c59378b71dc526f1f5a568b37d1354b0f80871ac18c114af42b/2d3bd82534bb6c6e0f4be1761b29b25bdcd65855a022513cb65f24ad3b25e41f?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27tinyllama-1.1b-chat-v0.3.Q4_K_M.gguf%3B+filename%3D%22tinyllama-1.1b-chat-v0.3.Q4_K_M.gguf%22%3B&Expires=1756030566&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1NjAzMDU2Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy82NS9iYi82NWJiMmI0NWVkM2U3YzU5Mzc4YjcxZGM1MjZmMWY1YTU2OGIzN2QxMzU0YjBmODA4NzFhYzE4YzExNGFmNDJiLzJkM2JkODI1MzRiYjZjNmUwZjRiZTE3NjFiMjli

In [15]:
!pip install llama-cpp-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.0 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl size=4422342 sha256=26bfed09b07230f0d2194a496f17072afec8f5efe3369ebc20dcb780ac422dba
  Stored in directory: /root/.cache/pip/wheels/90/82/ab/8784ee3fb99ddb07fd36a679ddbe63122cc07718f6c1eb3be8
Successfully built llama-cpp-python


In [21]:
import os

provider = os.getenv("MODEL_PROVIDER", "openai")

if provider == "openai":
    from openai import OpenAI
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
elif provider == "huggingface":
    from transformers import AutoModelForCausalLM, AutoTokenizer
    model_name = "gpt2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
elif provider == "llama":
    # Example for local LLaMA
    from llama_cpp import Llama
    model_path = "/content/tinyllama.gguf"
    model = Llama(model_path=model_path)
else:
    raise ValueError(f"Unknown MODEL_PROVIDER: {provider}")



In [23]:
import os

# Global variables to cache models for efficiency
hf_model = None
hf_tokenizer = None
llama_model = None

def generate_text(prompt, max_tokens=100):
    global hf_model, hf_tokenizer, llama_model

    if provider == "openai":
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content

    elif provider == "huggingface":
        # Load model once if not already loaded
        if hf_model is None or hf_tokenizer is None:
            from transformers import AutoModelForCausalLM, AutoTokenizer
            model_name = os.getenv("HF_MODEL_NAME", "gpt2")
            hf_tokenizer = AutoTokenizer.from_pretrained(model_name)
            hf_model = AutoModelForCausalLM.from_pretrained(model_name)

        inputs = hf_tokenizer(prompt, return_tensors="pt")
        outputs = hf_model.generate(**inputs, max_new_tokens=max_tokens)
        return hf_tokenizer.decode(outputs[0], skip_special_tokens=True)

    elif provider == "llama":
        # Load LLaMA model once if not already loaded
        if llama_model is None:
            from llama_cpp import Llama
            model_path = os.getenv("LLAMA_MODEL_PATH", "/content/tinyllama.gguf")
            llama_model = Llama(model_path=model_path)

        return llama_model(prompt, max_tokens=max_tokens)['choices'][0]['text']



In [24]:
provider = "llama"
prompt = "Explain the benefits of a steam bath."
answer = generate_text(prompt)
print(answer)


llama_model_loader: loaded meta data with 20 key-value pairs and 201 tensors from /content/tinyllama.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = py007_tinyllama-1.1b-chat-v0.3
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          llama.block_count u32              = 22
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 5632
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 64
llama_model_loader: - kv   7:                 llama.attention.head_count u32

 What are the health benefits of a steam bath? What are the health benefits of steam exposure? Explain the benefits of a steam bath in terms of the body's ability to detoxify itself. What are some of the health benefits of steam exposure, such as improving blood flow, reducing stress levels, and reducing the effects of pain? What are some of the potential risks of steam bathing, such as dehydration, headaches, and nausea
